In [4]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession. builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/29 07:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet


--2024-07-29 08:01:47--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.200, 54.230.209.140, 54.230.209.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 302633211 (289M) [application/x-www-form-urlencoded]
Saving to: ‘fhvhv_tripdata_2021-02.parquet’

fhvhv_tripdata_2021 100%[===================>] 288.61M  37.4MB/s    in 7.6s    

2024-07-29 08:01:56 (37.8 MB/s) - ‘fhvhv_tripdata_2021-02.parquet’ saved [302633211/302633211]



In [23]:
df = spark.read.option("header","true").parquet("fhvhv_tripdata_2021-02.parquet")

In [29]:
from pyspark.sql import types

In [24]:
df.schema #check column types and names 

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [56]:
schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(), True),
    types.StructField("dispatching_base_num", types.StringType(), True),
    types.StructField("originating_base_num", types.StringType(), True),
    types.StructField("request_datetime", types.TimestampType(), True),
    types.StructField("on_scene_datetime", types.TimestampType(), True),
    types.StructField("pickup_datetime", types.TimestampType(), True),
    types.StructField("dropoff_datetime", types.TimestampType(), True),
    types.StructField("PULocationID", types.LongType(), True),
    types.StructField("DOLocationID", types.LongType(), True),
    types.StructField("trip_miles", types.DoubleType(), True),
    types.StructField("trip_time", types.LongType(), True),
    types.StructField("base_passenger_fare", types.DoubleType(), True),
    types.StructField("tolls", types.DoubleType(), True),
    types.StructField("bcf", types.DoubleType(), True),
    types.StructField("sales_tax", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True),
    types.StructField("airport_fee", types.DoubleType(), True),
    types.StructField("tips", types.DoubleType(), True),
    types.StructField("driver_pay", types.DoubleType(), True),
    types.StructField("shared_request_flag", types.StringType(), True),
    types.StructField("shared_match_flag", types.StringType(), True),
    types.StructField("access_a_ride_flag", types.StringType(), True),
    types.StructField("wav_request_flag", types.StringType(), True),
    types.StructField("wav_match_flag", types.StringType(), True),

])

In [57]:
df = spark.read.option("header","true").schema(schema).parquet("fhvhv_tripdata_2021-02.parquet")

In [58]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [59]:
df = df.repartition(24) #break down the data in to 24 seperate partitions for better processing by spark cluster nodes

In [60]:

df.write.parquet("fhvhv/2021/02/data/") #Create 24 seperate parquet files in this directory 

In [63]:
!ls fhvhv/2021/02/data



_SUCCESS
part-00000-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00001-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00002-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00003-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00004-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00005-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00006-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00007-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00008-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00009-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00010-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00011-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00012-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00013-f4020195-ec01-4a10-8aac-7543b62aecd3-c000.snappy.parquet
part-00014-f4020195-ec01-4a10-8aac-7543